In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [92]:
test = pd.read_csv('test_nvPHrOx.csv')

In [93]:
train = pd.read_csv('train.csv')

In [94]:
sample_submission = pd.read_csv('sample_submission_poy1UIu.csv')

In [95]:
html_data = pd.read_csv('html_data.csv', nrows=5000)

In [96]:
html_data.head()

,Webpage_id,Html
0,1,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" xml..."
1,2,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" xml..."
2,3,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" xml..."
3,4,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" xml..."
4,5,"<!DOCTYPE html>\n<html lang=""en"" dir=""ltr"" xml..."


In [97]:
html_data.shape

(5000, 2)

In [98]:
#new_train = pd.merge(train, html_data, on='Webpage_id')
new_train = train

In [99]:
new_train.shape

(53447, 4)

In [100]:
len(train.Webpage_id.unique())

53447

In [101]:
html_data.Webpage_id.unique()

array([   1,    2,    3, ..., 4998, 4999, 5000])

In [102]:
from bs4 import BeautifulSoup

In [103]:
soup = BeautifulSoup(html_data.Html[0])

/home/parmardiwakar150/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/parmardiwakar150/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [104]:
soup.title.string

"Tecfidera, Gilenya and Aubagio's 3-way battle for MS share is about to heat up | FiercePharma"

In [105]:
def title(html):
    soup = BeautifulSoup(html)
    return soup.title.string

In [106]:
#new_train.Html = new_train.Html.apply(title)

In [107]:
new_train.head()

,Webpage_id,Domain,Url,Tag
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfider...,news
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipp...,news
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exe...,news
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-bi...,news
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-...,news


In [108]:
from nltk.corpus import stopwords
import string

In [109]:
def text_process(mess):
    nopunc = [char for char in mess if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    return [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [110]:
#X = new_train[['Domain', 'Html']]
X = new_train.Domain
y = new_train.Tag

In [113]:
#X.Domain = X.Domain.apply(lambda x: x.split('.')[1])
X = X.apply(lambda x: x.split('.')[1])

In [114]:
X.head()

0    fiercepharma
1    fiercepharma
2    fiercepharma
3    fiercepharma
4    fiercepharma
Name: Domain, dtype: object

In [115]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, TfidfTransformer

In [116]:
#tfidf_vectorizer_domain = TfidfVectorizer(analyzer=text_process).fit(X.Domain)
tfidf_vectorizer_domain = TfidfVectorizer(analyzer=text_process).fit(X)

In [117]:
#tfidf_vectorizer_html = TfidfVectorizer(analyzer=text_process).fit(X.Html)

In [118]:
#X = tfidf_vectorizer_domain.transform(X.Domain)
#tfidf_vectorizer_domain.transform(X.Domain).shape
X = tfidf_vectorizer_domain.transform(X)

In [119]:
tfidf_transformer_domain = TfidfTransformer().fit(X)

In [120]:
X = tfidf_transformer_domain.transform(X)

In [121]:
X

<53447x1768 sparse matrix of type '<class 'numpy.float64'>'
	with 53416 stored elements in Compressed Sparse Row format>

In [122]:
#X.Html = tfidf_vectorizer_html.transform(X.Html)

In [123]:
from sklearn.tree import DecisionTreeClassifier

In [124]:
clf = DecisionTreeClassifier()

In [125]:
from sklearn.model_selection import train_test_split

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=101)

In [127]:
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [128]:
pred = clf.predict(X_test)

In [129]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [130]:
print(classification_report(pred, y_test))

                precision    recall  f1-score   support

clinicalTrials       0.91      0.96      0.93       267
   conferences       0.76      0.93      0.84       403
         forum       0.83      0.95      0.88       416
    guidelines       0.69      0.91      0.78       112
          news       0.62      0.83      0.71       557
        others       0.87      0.66      0.75      2248
       profile       0.56      0.93      0.70       315
   publication       0.87      0.78      0.82       865
        thesis       0.86      0.99      0.92       162

   avg / total       0.81      0.79      0.79      5345



In [131]:
test.head()

,Webpage_id,Domain,Url
0,31,isrctn.com,http://www.isrctn.com/ISRCTN57801413
1,32,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...
2,33,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...
3,34,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...
4,35,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...


In [132]:
feat = test.Domain

In [133]:
feat = feat.apply(lambda x: x.split('.')[1])

In [134]:
feat[:5]

0                       com
1    clinicaltrialsregister
2    clinicaltrialsregister
3    clinicaltrialsregister
4    clinicaltrialsregister
Name: Domain, dtype: object

In [135]:
np.unique(feat)

array(['07', '13', '139', ..., 'zacks', 'zain', 'zoomlion'], dtype=object)

In [136]:
feat = tfidf_vectorizer_domain.transform(feat)

In [137]:
feat = tfidf_transformer_domain.transform(feat)

In [138]:
test_pred = clf.predict(feat)

In [139]:
test['Tag'] = test_pred

In [140]:
test.head()

,Webpage_id,Domain,Url,Tag
0,31,isrctn.com,http://www.isrctn.com/ISRCTN57801413,others
1,32,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,others
2,33,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,others
3,34,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,others
4,35,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,others


In [141]:
submission = test[['Webpage_id', 'Tag']]

In [142]:
sample_submission.head()

,Webpage_id,Tag
0,31,news
1,32,news
2,33,news
3,34,news
4,35,news


In [143]:
submission.to_csv('domain_only.csv', index=False)